In [1]:
import getkey
import json
import pandas as pd
import folium
import glob
import branca
import time
from folium.plugins import HeatMap
from geopy.distance import geodesic


In [3]:
citylist = ['YilanCounty', 'HsinchuCounty', 'ChanghuaCounty', 'NantouCounty', 'YunlinCounty', 'PingtungCounty', 'TaitungCounty', 'Keelung', 'Hsinchu', 'Chiayi', 'Taipei', 'Kaohsiung', 'NewTaipei', 'Taichung', 'Tainan', 'Taoyuan']


In [4]:
for city in citylist:
    time.sleep(0.5)
    getkey.getjson(f'https://tdx.transportdata.tw/api/basic/v2/Road/Traffic/CCTV/City/{city}?%24top=100&%24format=JSON', f'test-{city}')

In [16]:
getkey.getjson(f'https://tdx.transportdata.tw/api/basic/v2/Road/Traffic/CCTV/Freeway?%24top=50&%24format=JSON', f'test-highway')

In [5]:
column = ['CCTVID', 'LinkID', 'VideoStreamURL', 'PositionLon', 'PositionLat' ,'RoadName', 'SurveillanceDescription']
df = pd.DataFrame(columns=column)
for city in citylist:        
    with open(f'data/test-{city}.json', 'r', encoding='utf-8') as f:
        testTV = json.load(f)['CCTVs']
    #print(testTV)
    city_data = {}
    for TV in testTV:
        #print(TV)
        for col in column:
            if col in TV:
                
                city_data[col] = TV[col]
            else:
                print(city, col)
        #print(city_data)
        df = pd.concat([df, pd.DataFrame([city_data])], ignore_index=True)
            
        


YilanCounty RoadName
YilanCounty RoadName
YilanCounty RoadName
YilanCounty RoadName
YilanCounty RoadName
YilanCounty RoadName
YilanCounty RoadName
YilanCounty RoadName
YilanCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
HsinchuCounty RoadName
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription
NantouCounty SurveillanceDescription

C:\Users\good1\AppData\Local\Temp\ipykernel_22496\1233470955.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([city_data])], ignore_index=True)


PingtungCounty SurveillanceDescription
PingtungCounty SurveillanceDescription
PingtungCounty SurveillanceDescription
PingtungCounty SurveillanceDescription
PingtungCounty SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Chiayi SurveillanceDescription
Taipei SurveillanceDescription
Taipei SurveillanceDescription

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df

In [19]:
#初始化公車地圖
Taiwan_map = folium.Map(location=[23.800983, 120.772569], tiles='openstreetmap', zoom_start=7.3)


fg2 = folium.FeatureGroup(name=f'city-group',show=False).add_to(Taiwan_map)

for row in df.iloc:
    
    f = branca.element.Figure()

# Let's put the figure into an IFrame.
    iframe = branca.element.IFrame(html= f"""
                                <h2>{row['RoadName'] if not pd.isnull(row['RoadName']) else  row['SurveillanceDescription']}</h2>
                                <iframe src="{row['VideoStreamURL']}" width="600" height="300" frameborder="0" allowfullscreen></iframe>""",
                                width=600, height=400)
    f.add_to(iframe)

    # Let's put the IFrame in a Popup
    popup = folium.Popup(iframe, max_width=2500, lazy=True)

    mm = folium.Marker(location=(row['PositionLat'], row['PositionLon']),#座標
                       popup=popup, #車牌
                       icon=folium.Icon(color='orange', icon='facetime-video'),
    )
    mm.add_to(fg2)
    #Taiwan_map.add_child(mm)

#只需要打一次
#folium.FitOverlays(padding=10).add_to(Taiwan_map)
#folium.LayerControl().add_to(Taiwan_map)

#Taiwan_map


In [20]:
bus_data = ""
with open('data/test-freq.json', 'r', encoding='utf-8') as f:
    bus_data = json.load(f)
    
target_location = (25.033964, 121.564472)  # 台北101的經緯度
radius = 2.0  # 公里

filtered_data = []
for item in bus_data:
    bus_location = (item['BusPosition']['PositionLat'], item['BusPosition']['PositionLon'])
    if geodesic(target_location, bus_location).km <= radius:
        filtered_data.append((bus_location[0], bus_location[1]))
        
# 創建地圖

heatmap_layer = folium.FeatureGroup(name='Bus Traffic Heatmap', show=False)
HeatMap(filtered_data).add_to(heatmap_layer)

# 將熱圖子層添加到地圖
heatmap_layer.add_to(Taiwan_map)

#layer start
folium.FitOverlays(padding=10).add_to(Taiwan_map)
folium.LayerControl().add_to(Taiwan_map)

Taiwan_map

In [ ]:
Taiwan_map = folium.Map(location=[23.800983, 120.772569], tiles='openstreetmap', zoom_start=7.3)


fg2 = folium.FeatureGroup(name='enty',show=False).add_to(Taiwan_map)
folium.Marker((52, 5.1)).add_to(fg2)
folium.FitOverlays(padding=10).add_to(Taiwan_map)
folium.LayerControl().add_to(Taiwan_map)

Taiwan_map


In [21]:
Taiwan_map.save('test.html')

## 高速公路CCTV

In [8]:
getkey.getjson(f'https://tdx.transportdata.tw/api/basic/v2/Road/Traffic/CCTV/Freeway?%24top=200&%24format=JSON', f'test-highway')

In [9]:

with open(f'data/test-highway.json', 'r', encoding='utf-8') as f:
        highTV = json.load(f)['CCTVs']

In [12]:
df = pd.DataFrame(highTV)
df.head()

,CCTVID,SubAuthorityCode,LinkID,VideoStreamURL,LocationType,PositionLon,PositionLat,RoadID,RoadName,RoadClass,RoadDirection,RoadSection,LocationMile
0,CCTV-N3K-N-0.270-M,NFB-NR,3000260100118C,https://cctvn.freeway.gov.tw/abs2mjpg/bmjpg?ca...,1,121.736574,25.152457,300026,台2己線,3.0,E,"{'Start': '德安路交流道', 'End': '基隆港交流道'}",0K+270
1,CCTV-N1H-N-57.738-M,NFB-NR,00001B0105700H,https://cctvn.freeway.gov.tw/abs2mjpg/bmjpg?ca...,1,121.234360,24.985104,00001B,國道1號五股楊梅高架道路,0.0,N,"{'Start': '中壢轉接一交流道', 'End': '機場系統交流道'}",57K+738
2,CCTV-N1H-S-53.710-M,NFB-NR,00001B0005300H,https://cctvn.freeway.gov.tw/abs2mjpg/bmjpg?ca...,1,121.262390,25.010677,00001B,國道1號五股楊梅高架道路,0.0,S,"{'Start': '機場系統交流道', 'End': '中壢轉接一交流道'}",53K+710
3,CCTV-N1-N-345.000-M,NFB-SR,0000100134500E,https://cctvs.freeway.gov.tw/live-view/mjpg/vi...,1,120.308920,22.826277,000010,國道1號,0.0,N,"{'Start': '岡山交流道', 'End': '高科交流道'}",345K+000
4,CCTV-N3-S-105.700-M,NFB-NR,0000300010500O,https://cctvn.freeway.gov.tw/abs2mjpg/bmjpg?ca...,1,120.940217,24.753328,000030,國道3號,0.0,S,"{'Start': '茄苳交流道', 'End': '香山交流道'}",105K+700


In [15]:
Taiwan_map = folium.Map(location=[23.800983, 120.772569], tiles='openstreetmap', zoom_start=7.3)

for row in df.iloc:
    
    f = branca.element.Figure()

# Let's put the figure into an IFrame.
    iframe = branca.element.IFrame(html= f"""
                                <h2>{row['RoadSection']['Start']}-{row['RoadSection']['End']}</h2>
                                <iframe src="{row['VideoStreamURL']}" width="600" height="300" frameborder="0" allowfullscreen></iframe>""",
                                width=600, height=400)
    f.add_to(iframe)

    # Let's put the IFrame in a Popup
    popup = folium.Popup(iframe, max_width=2500, lazy=True)

    mm = folium.Marker(location=(row['PositionLat'], row['PositionLon']),#座標
                       popup=popup, #車牌
                       icon=folium.Icon(color='orange', icon='facetime-video'),
    )
    Taiwan_map.add_child(mm)
Taiwan_map